In [1]:
%matplotlib inline
# !pip install stargazer
# !pip install scipy
# !pip install plotly_express
# !pip install pandas-profiling
# !pip install linearmodels
# !pip install sklearn

import scipy
import numpy as np
import pandas as pd
import pandas.io.formats.style
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from IPython.display import HTML
import linearmodels as lm
from linearmodels import PanelOLS
import warnings
warnings.filterwarnings("ignore")
import math
# import sklearn


<a class="anchor" id="8. Difference in differences implementation"></a>


# cash flow is already level data   and there is no infor for No.of bank relations 

In [2]:
path="/Users/effie/Desktop/term3/research_project/data/"
data=pd.read_csv(path+"firm_clean/ready_for_regression.csv")

In [3]:
data

,Unnamed: 0,BvDID_firm,year,plz_firm,ort_,regionalkey_,bundesland,BvDID_bank_1,postcode_bank_1,data2025_1,...,shfdlev,random,random_sample_indicator,small,year_post,bank_ty_1,bank_ty_2,bank_ty_3,bank_affected,bank_affected_degree
0,1,DE2010041272,2006.0,13407,"Berlin, Stadt",11,Berlin,DE13190,60311,265857.0,...,3.397223,0.061524,0.575035,1,0,0,0,1,1,1
1,2,DE2010041272,2007.0,13407,"Berlin, Stadt",11,Berlin,DE13190,60311,302841.0,...,2.990665,0.819242,0.575035,1,0,0,0,1,1,1
2,3,DE2010041272,2008.0,13407,"Berlin, Stadt",11,Berlin,DE13190,60311,359251.0,...,2.685565,0.925363,0.575035,1,0,0,0,1,1,1
3,4,DE2010041272,2009.0,13407,"Berlin, Stadt",11,Berlin,DE13190,60311,605765.0,...,2.495652,0.128524,0.575035,1,1,0,0,1,1,1
4,5,DE2010041272,2010.0,13407,"Berlin, Stadt",11,Berlin,DE13190,60311,623634.0,...,2.437330,0.728537,0.575035,1,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21066,21067,DE8350193694,2008.0,84034,"Landshut, Stadt",9261,Bayern,DE13732,84028,3076.0,...,81.250750,0.406138,0.552057,1,0,1,0,0,1,1
21067,21068,DE8350193694,2009.0,84034,"Landshut, Stadt",9261,Bayern,DE13732,84028,3135.0,...,77.696000,0.359884,0.552057,1,1,1,0,0,1,1
21068,21069,DE8350193694,2010.0,84034,"Landshut, Stadt",9261,Bayern,DE13732,84028,3073.0,...,78.190000,0.981670,0.552057,1,1,1,0,0,1,1
21069,21070,DE8350193694,2011.0,84034,"Landshut, Stadt",9261,Bayern,DE13732,84028,3274.0,...,71.727050,0.503510,0.552057,1,1,1,0,0,1,1


In [4]:
data=data.drop(columns=['Unnamed: 0'])
data.rename(columns={"bank_affected":"firm_affected"},inplace=True)
data["shfdlev"]=np.exp(data["shfd"])
data["groslev"]=np.exp(data["gros"])
data["firm_affected"].value_counts()


1    14560
0     6511
Name: firm_affected, dtype: int64

In [5]:
data.columns

Index(['BvDID_firm', 'year', 'plz_firm', 'ort_', 'regionalkey_', 'bundesland',
       'BvDID_bank_1', 'postcode_bank_1', 'data2025_1', 'data2075_1',
       'data11750_1', 'data30680_1', 'data30690_1', 'data38250_1',
       'bank_type_1', 'liquidity_1', 'BvDID_bank_2', 'postcode_bank_2',
       'data2025_2', 'data2075_2', 'data11750_2', 'data30680_2', 'data30690_2',
       'data38250_2', 'bank_type_2', 'liquidity_2', 'BvDID_bank_3',
       'postcode_bank_3', 'data2025_3', 'data2075_3', 'data11750_3',
       'data30680_3', 'data30690_3', 'data38250_3', 'bank_type_3',
       'liquidity_3', 'fias', 'tfas', 'cash', 'toas', 'shfd', 'ncli', 'ltdb',
       'oncl', 'prov', 'culi', 'loan', 'cred', 'wkca', 'ncas', 'empl', 'opre',
       'turn', 'gros', 'staf', 'depr', 'cf', 'rcem', 'rtas',
       'dateinc_year_firm', 'sizecat', 'sector', 'adequacy', 'hausbank',
       'KID_firm', 'empllev', 'staflev', 'fiaslev', 'tfaslev', 'cashlev',
       'toaslev', 'provlev', 'ncaslev', 'turnlev', 'groslev', '

<a class="anchor" id="8. Difference in differences implementation"></a>

---
# Difference in differences implementation
---

In [6]:
def did_est(variable,df):
    '''
    arguments:variable of interest, dataset
    return: 2x2 dif-in-dif table
    '''
    
    # affected Before and after
    affected_before = round(df.loc[(df['firm_affected']==1) & (df['year_post']!=1),variable].mean(),2)
    affected_after  = round(df.loc[(df['firm_affected']==1) & (df['year_post']==1),variable].mean(),2)

    # unaffected Before and after
    unaffected_before = round(df.loc[(df['firm_affected']!=1) & (df['year_post']!=1),variable].mean(),2)
    unaffected_after  = round(df.loc[(df['firm_affected']!=1) & (df['year_post']==1),variable].mean(),2)
    
    did= pd.DataFrame(data=np.array([[affected_after, affected_before,affected_after - affected_before],
    [unaffected_after, unaffected_before, unaffected_after - unaffected_before]]), index=['affected','unaffected'],columns=['after','before','$\Delta$'])
    
    return did

In [7]:
did_empl=did_est('empl',data)

In [8]:
print("Dif-in-dif estimate for empl is", round(did_empl['$\Delta$'][0]-did_empl['$\Delta$'][1],2))

Dif-in-dif estimate for empl is -0.05


In [9]:
did_staf=did_est('staf',data)

In [10]:
print("Dif-in-dif estimate for staff expense is", round(did_staf['$\Delta$'][0]-did_staf['$\Delta$'][1],2))

Dif-in-dif estimate for staff expense is -0.04


<a class="anchor" id="9. Regression-adjusted models"></a>

---
#  Regression-adjusted models
---

 The DiD estimator can easily be implemented using regression. This is
a convenient way to obtain estimates and the corresponding standard errors in one step.

Consider the DiD linear regression model:  

$$employment_{i,t} = \beta_1 (Post_t \times affected_i) + \beta_2 X_{it} + \beta_3 firm_i +\beta_4 time_i + \varepsilon_{i,t}$$

In [11]:
model1 = smf.ols(formula = "empl ~ firm_affected*year_post", data = data).fit()

In [12]:
print("Dif-in-dif coefficient estimate for no of employee is ",round(model1.params[3],3))

Dif-in-dif coefficient estimate for no of employee is  -0.051


In [13]:
model2 = smf.ols(formula = "staf ~ firm_affected*year_post", data = data).fit()

In [14]:
print("Dif-in-dif coefficient estimate for staff expense is ",round(model2.params[3],3))

Dif-in-dif coefficient estimate for staff expense is  -0.036


  

###  DiD  only include  fixed effects 
$$employment_{i,t} = \beta_1 (post_t \times affected_i) +u_i+v_t+ \varepsilon_{i,t}$$

where $u_i$ isw a firm fixed effect and $v_t$ is a time fixed effect.



In [15]:
data["affected_after"] = data.firm_affected*data.year_post
df_fe = data.set_index(['BvDID_firm','year'])
df_fe

plz_firm             ort_  regionalkey_ bundesland  \
BvDID_firm   year                                                         
DE2010041272 2006.0     13407    Berlin, Stadt            11     Berlin   
             2007.0     13407    Berlin, Stadt            11     Berlin   
             2008.0     13407    Berlin, Stadt            11     Berlin   
             2009.0     13407    Berlin, Stadt            11     Berlin   
             2010.0     13407    Berlin, Stadt            11     Berlin   
...                       ...              ...           ...        ...   
DE8350193694 2008.0     84034  Landshut, Stadt          9261     Bayern   
             2009.0     84034  Landshut, Stadt          9261     Bayern   
             2010.0     84034  Landshut, Stadt          9261     Bayern   
             2011.0     84034  Landshut, Stadt          9261     Bayern   
             2012.0     84034  Landshut, Stadt          9261     Bayern   

                    BvDID_bank_1  postcode_bank_1  data2025_1  data2075_1  \
BvDID_firm   year                                                           
DE2010041272 2006.0      DE13190            60311    265857.0     79992.0   
             2007.0      DE13190            60311    302841.0    103003.0   
             2008.0      DE13190            60311    359251.0    126692.0   
             2009.0      DE13190            60311    605765.0    177120.0   
             2010.0      DE13190            60311    623634.0    370294.0   
...                          ...              ...         ...         ...   
DE8350193694 2008.0      DE13732            84028      3076.0      1154.0   
             2009.0      DE13732            84028      3135.0      1017.0   
             2010.0      DE13732            84028      3073.0       875.0   
             2011.0      DE13732            84028      3274.0       548.0   
             2012.0      DE13732            84028      3268.0       564.0   

                     data11750_1  data30680_1  ...    random  \
BvDID_firm   year                              ...             
DE2010041272 2006.0     253930.0          NaN  ...  0.061524   
             2007.0     291005.0         6.90  ...  0.819242   
             2008.0     340248.0        10.10  ...  0.925363   
             2009.0     582393.0        10.50  ...  0.128524   
             2010.0     599549.0        11.90  ...  0.728537   
...                          ...          ...  ...       ...   
DE8350193694 2008.0       2826.0          NaN  ...  0.406138   
             2009.0       2857.0          NaN  ...  0.359884   
             2010.0       2753.0          NaN  ...  0.981670   
             2011.0       2819.0          NaN  ...  0.503510   
             2012.0       2785.0        21.27  ...  0.724514   

                     random_sample_indicator small  year_post bank_ty_1  \
BvDID_firm   year                                                         
DE2010041272 2006.0                 0.575035     1          0         0   
             2007.0                 0.575035     1          0         0   
             2008.0                 0.575035     1          0         0   
             2009.0                 0.575035     1          1         0   
             2010.0                 0.575035     1          1         0   
...                                      ...   ...        ...       ...   
DE8350193694 2008.0                 0.552057     1          0         1   
             2009.0                 0.552057     1          1         1   
             2010.0                 0.552057     1          1         1   
             2011.0                 0.552057     1          1         1   
             2012.0                 0.552057     1          1         1   

                     bank_ty_2  bank_ty_3  firm_affected  \
BvDID_firm   year                                          
DE2010041272 2006.0          0          1              1   
             2007.0          0          1             

In [16]:
#for empl
mod = lm.PanelOLS.from_formula('empl ~ affected_after + EntityEffects +TimeEffects ',df_fe)
clfe_empl = mod.fit(cov_type = 'clustered',
cluster_entity = True, cluster_time=True)
print(clfe_empl)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   empl   R-squared:                        0.0009
Estimator:                   PanelOLS   R-squared (Between):             -0.0064
No. Observations:               21071   R-squared (Within):              -0.0054
Date:                Mon, Dec 20 2021   R-squared (Overall):             -0.0064
Time:                        22:40:45   Log-likelihood                    890.28
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      14.724
Entities:                        3944   P-value                           0.0001
Avg Obs:                       5.3425   Distribution:                 F(1,17119)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             6.7601
                            

In [17]:
#for staff expense
mod = lm.PanelOLS.from_formula('staf ~ affected_after + EntityEffects +TimeEffects ',df_fe)
clfe_staf = mod.fit(cov_type = 'clustered',
cluster_entity = True, cluster_time=True)
print(clfe_staf)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   staf   R-squared:                      5.61e-05
Estimator:                   PanelOLS   R-squared (Between):             -0.0004
No. Observations:               16429   R-squared (Within):              -0.0033
Date:                Mon, Dec 20 2021   R-squared (Overall):             -0.0005
Time:                        22:40:46   Log-likelihood                    2028.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      0.7391
Entities:                        3247   P-value                           0.3900
Avg Obs:                       5.0597   Distribution:                 F(1,13174)
Min Obs:                       1.0000                                           
Max Obs:                       8.0000   F-statistic (robust):             0.4208
                            

<a class="anchor" id="9. Regression-adjusted models"></a>

---
#  replication of table 3

---

### Adding controls---Table 3 column 1    empl

The regression formulation of DiD also allows to include additional control variables.  
In a DiD you can control for other covariates that may differ between the groups, and the main reason for doing this is to have a more robust identification and decreasing the variance of the regression coefficient estimates.  

The DiD linear regression model becomes:

$$employment_{i,t} = \beta_1 (Post_t \times affected_i) + \beta_2 X_{it} + \beta_3 firm_i +\beta_4 time_i + \varepsilon_{i,t}$$
 

In [18]:
# data=data.dropna(axis=0)
name=["shfdlev", "groslev", "cf"]
np.any(~np.isfinite(data[name]),axis=0)

shfdlev    False
groslev     True
cf          True
dtype: bool

In [19]:
model = smf.ols(formula = "empl ~ firm_affected*year_post + firm_affected + year_post + toas + turn + shfdlev+ groslev+ cf ", data = data).fit()
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-9.0875,0.869,-10.452,0.000,-10.807,-7.368
firm_affected,-0.1341,0.177,-0.757,0.450,-0.484,0.216
year_post,-0.2058,0.199,-1.033,0.304,-0.600,0.188
firm_affected:year_post,0.0181,0.221,0.082,0.935,-0.420,0.456
toas,0.2553,0.075,3.411,0.001,0.107,0.403
turn,0.5862,0.058,10.053,0.000,0.471,0.702
shfdlev,2.857e-10,7.34e-10,0.389,0.698,-1.16e-09,1.74e-09
groslev,-1.674e-10,1.07e-09,-0.156,0.876,-2.29e-09,1.96e-09
cf,2.799e-10,1.01e-09,0.278,0.781,-1.71e-09,2.27e-09


### Adding controls and fixed effect---Table 3 column 2   empl



In [20]:
mod = lm.PanelOLS.from_formula('empl ~ affected_after + EntityEffects +TimeEffects + toas +turn + shfdlev + groslev + cf',df_fe)
# print(mod.params)
# print('\n')
# print(mod.pvalues)
didfe_empl = mod.fit(cov_type = 'clustered',
cluster_entity = True, cluster_time=True)
print(didfe_empl)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   empl   R-squared:                        0.7015
Estimator:                   PanelOLS   R-squared (Between):             -4.6141
No. Observations:                 146   R-squared (Within):               0.7186
Date:                Mon, Dec 20 2021   R-squared (Overall):             -4.1542
Time:                        22:40:46   Log-likelihood                    85.140
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      37.988
Entities:                          37   P-value                           0.0000
Avg Obs:                       3.9459   Distribution:                    F(6,97)
Min Obs:                       1.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             3.3606
                            

### Adding controls---Table 3 column 3    staff



In [21]:
model = smf.ols(formula = "staf ~ firm_affected*year_post + firm_affected + year_post + toas +turn + shfdlev + groslev + cf", data = data).fit()
model.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.4418,0.856,0.516,0.607,-1.251,2.135
firm_affected,0.1020,0.173,0.589,0.557,-0.241,0.445
year_post,-0.0157,0.191,-0.082,0.935,-0.393,0.362
firm_affected:year_post,-0.1496,0.210,-0.713,0.477,-0.565,0.265
toas,0.5000,0.072,6.901,0.000,0.357,0.643
turn,0.4037,0.053,7.608,0.000,0.299,0.509
shfdlev,6.931e-10,6.64e-10,1.044,0.298,-6.2e-10,2.01e-09
groslev,-6.377e-10,9.58e-10,-0.666,0.507,-2.53e-09,1.26e-09
cf,1.792e-10,8.94e-10,0.201,0.841,-1.59e-09,1.95e-09


### Adding controls---Table 3 column 4    staff



In [22]:
mod = lm.PanelOLS.from_formula('staf ~ affected_after + EntityEffects +TimeEffects + toas +turn + shfdlev + groslev + cf',df_fe)
didfe_staf = mod.fit(cov_type = 'clustered',
cluster_entity = True, cluster_time=True)
print(didfe_staf)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   staf   R-squared:                        0.3571
Estimator:                   PanelOLS   R-squared (Between):              0.9118
No. Observations:                 142   R-squared (Within):               0.3702
Date:                Mon, Dec 20 2021   R-squared (Overall):              0.9118
Time:                        22:40:47   Log-likelihood                    59.214
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      8.7018
Entities:                          36   P-value                           0.0000
Avg Obs:                       3.9444   Distribution:                    F(6,94)
Min Obs:                       1.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             4.6480
                            

## Heteroskedastic-Corrected and clustered standard errors 




In [23]:
# empl 
cluster_mod = smf.ols(formula='empl ~ affected_after', data=df_fe).fit(cov_type='nonrobust') 
fe_groups = df_fe.copy()
for i in ['empl', 'firm_affected',"year_post"]:
    fe_groups = fe_groups[pd.notnull(fe_groups[i])]# drop any entries with missing data from the model
result=cluster_mod.get_robustcov_results(cov_type='cluster',groups=fe_groups['sizecat'])#adjust SEs
result.params
print(f'Dif-in-dif coefficient for empl with standard errors clustered by sizecat: coeff is {result.params[1]:.3f} and std.err. is {result.bse[1]:.3f} --> pvalue {result.pvalues[1]:.3f}')

Dif-in-dif coefficient for empl with standard errors clustered by sizecat: coeff is 0.038 and std.err. is 0.019 --> pvalue 0.143


In [24]:
# staf
cluster_mod = smf.ols(formula='staf ~ affected_after', data=df_fe).fit(cov_type='nonrobust') 
fe_groups = df_fe.copy()
for i in ['staf', 'firm_affected',"year_post"]:
    fe_groups = fe_groups[pd.notnull(fe_groups[i])]# drop any entries with missing data from the model
result=cluster_mod.get_robustcov_results(cov_type='cluster',groups=fe_groups['sizecat'])#adjust SEs
result.params
print(f'Dif-in-dif coefficient for staf with standard errors clustered by sizecat: coeff is {result.params[1]:.3f} and std.err. is {result.bse[1]:.3f} --> pvalue {result.pvalues[1]:.3f}')

Dif-in-dif coefficient for staf with standard errors clustered by sizecat: coeff is 0.112 and std.err. is 0.011 --> pvalue 0.002
